In [1]:
import pandas as pd
import math
import random
import warnings# xoá cảnh báo
import numpy as np

class dataLoader:
    def __init__(self, data_url, label):
        # xoá cảnh báo
        warnings.filterwarnings('ignore')
        pd.options.mode.chained_assignment = None
        
        #kiểm tra đuôi của url truyền vào
        if data_url.endswith('.csv'):
            self.data = pd.read_csv(data_url)
        elif data_url.endswith('.xlsx'):
            self.data = pd.read_excel(data_url)
        else:
            raise ValueError("Unsupported file format. Please provide a CSV or XLSX file.")
        
        self.label = label
        
        #kiểm tra label có trong dataset hay không
        if self.label not in self.data.columns.tolist():
            raise Exception("label not in dataset")
        
    # hàm tách biến độc lập và biến phụ thuộc
    def detachedData(self, data):
        Y = data[self.label]

        X = data.drop(columns=[self.label])

        return X,Y

    def hold_out(self, train_per = 0.7, val_per = 0.3):             
        self.train_data = self.data.iloc[0: math.floor(train_per * len(self.data))] #dòng 0 -> dòng thứ 70% của dataset
            
        self.val_data = self.data.iloc[len(self.train_data) :] # các dòng còn lại
        
        self.train_data.X, self.train_data.Y = self.detachedData(self.train_data) #tách thành 2 biến X, Y
        
        self.val_data.X, self.val_data.Y = self.detachedData(self.val_data) #tách thành 2 biến X, Y
        
    def kfold_split(self, K, shuffle=True, random_state=42):
        if shuffle:  # trộn các dòng của dữ liệu          
            indices = list(self.data.index) # lấy vị trí các dòng
            random.seed(random_state) # giúp các số random có thể lặp lại 
            random.shuffle(indices) # trộn vị trí
            self.data = self.data.iloc[indices].reset_index(drop=True) # cập nhập dữ liệu trên từng dòng sau khi trộn
            # chưa hiểu cách chạy self.data.iloc[indices].reset_index(drop=True)

        fold_sizes = np.full(K, len(self.data) // K, dtype=int) # tạo danh sách K phần tử chứa size của từng fold
        fold_sizes[: len(self.data) % K] += 1 # + 1 vào từ phần tử nếu data bị lẻ

        current = 0
        folds = []
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size # vị trí bắt đầu đến kết thúc <=> 1 fold
            folds.append(self.data.iloc[start:stop]) # tạo danh sách chứa K fold
            current = stop

        return folds
    
    def K_fold(self, K = 5):
        folds = self.kfold_split(K, shuffle=True)  # Tách từng fold cho K

        self.val_data = []
        self.train_data = []

        for i in range(K):
            self.val_data.append(folds[i])  # Chọn fold[i] làm tập validation

            train_j = pd.DataFrame()  # Khởi tạo lại tập huấn luyện trong mỗi vòng lặp
            for j in range(K):
                if i != j: # kiểm tra nếu trùng tập test thì không lấy
                    train_j = pd.concat([train_j, folds[j]], axis=0, ignore_index=True) # gộp các tập còn lại thành train_set
            self.train_data.append(train_j)  # Thêm train_j vào self.train_data

        for i in range(len(self.train_data)):
            self.train_data[i].X, self.train_data[i].Y = self.detachedData(self.train_data[i]) # tách thành 2 biến X, Y
            
            self.val_data[i].X, self.val_data[i].Y = self.detachedData(self.val_data[i])
            
            
    def LOO(self):
        self.K_fold(len(self.data))

In [2]:
import plotly.figure_factory as ff
import plotly.graph_objects as go
class clustering:
    def __init__(self, data): 
        self.data = np.array(data)
    
    def eculidean(self, a, b):#công thức eculidean distance
        return np.sqrt(np.sum((a-b)**2))
    
    def eculidean_matrix_distance(self):#Lập ra ma trận đối xứng qua đường chéo chính
        coordinates_points = self.data.shape[0]
        d_matrix = np.zeros((coordinates_points,coordinates_points))
        for i in range(coordinates_points):
            for j in range(i+1,coordinates_points):
                d_matrix[i,j] = self.eculidean(self.data[i],self.data[j])
                d_matrix[j,i] = d_matrix[i,j]
        return d_matrix
    
    def print_distance_matrix(self, d_matrix):
        print("Distance Matrix:")
        print(d_matrix)
        
    def single_linkage(self, cl1, cl2,d_matrix):
        min_d = np.inf # Khởi tạo khoảng cách nhỏ nhất là vô cực
        for i in cl1: # Duyệt qua tất cả các điểm trong cụm 1
            for j in cl2: 
                if d_matrix[i,j]<min_d: # Nếu khoảng cách giữa điểm i trong cl1 và điểm j trong cl2 nhỏ hơn
                    min_d=d_matrix[i,j] # Cập nhật khoảng cách nhỏ nhất
        return min_d
    
    def agglomerative_clustering(self):
        self.clusters = [] # Khởi tạo danh sách để lưu các cụm
        for i in range(len(self.data)):
            self.clusters.append([i]) # Tạo các cụm riêng biệt
        d_matrix  = self.eculidean_matrix_distance()
        self.merge = [] # Tạo để lưu thông tin các cụm đã gộp
        self.print_distance_matrix(d_matrix)
        while len(self.clusters) > 1: # While cho đến khi chỉ còn một cụm
            min_d = np.inf
            clusters_to_merge = (0, 0)
            for i in range(len(self.clusters)):
                for j in range(i + 1, len(self.clusters)):
                    dist_cluster = self.single_linkage(self.clusters[i], self.clusters[j], d_matrix)
                    if dist_cluster < min_d:
                        min_d = dist_cluster
                        clusters_to_merge = (i, j)
            self.merge.append((clusters_to_merge[0], clusters_to_merge[1], min_d))

            #show chi tiết gộp cụm
            print(f"Clusters: {self.clusters}")
            print(f"Gộp cụm {clusters_to_merge[0]} và {clusters_to_merge[1]} với d= {min_d:.2f}")
            
            # Tạo cụm mới = gộp hai cụm có khoảng cách nhỏ nhất và loại bỏ 2 cụm vừa mới được gộp
            new_cluster = self.clusters[clusters_to_merge[0]] + self.clusters[clusters_to_merge[1]]
            self.clusters.pop(clusters_to_merge[1])
            self.clusters.pop(clusters_to_merge[0])
            self.clusters.append(new_cluster)

            #Các cụm đã gộp
            print(f"Clusters đã gộp: {self.clusters}\n")
    def plot_dendrogram_with_heatmap(self):
        d_matrix = self.eculidean_matrix_distance()
        heat_data = d_matrix
        
        fig = ff.create_dendrogram(heat_data, orientation='bottom')
        for i in range(len(fig['data'])):
            fig['data'][i]['yaxis'] = 'y2'
        
        dendro_side = ff.create_dendrogram(heat_data, orientation='right')
        for i in range(len(dendro_side['data'])):
            dendro_side['data'][i]['xaxis'] = 'x2'
        
        for data in dendro_side['data']:
            fig.add_trace(data)
        
        dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
        dendro_leaves = list(map(int, dendro_leaves))
        heat_data = heat_data[dendro_leaves, :]
        heat_data = heat_data[:, dendro_leaves]
        
        heatmap = [
            go.Heatmap(
                x=dendro_leaves,
                y=dendro_leaves,
                z=heat_data,
                colorscale='Blues'
            )
        ]
        
        heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
        heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']
        
        for data in heatmap:
            fig.add_trace(data)
        
        fig.update_layout({'width': 800, 'height': 800,
                           'showlegend': False, 'hovermode': 'closest',
                          })
        
        fig.update_layout(xaxis={'domain': [.15, 1],
                                 'mirror': False,
                                 'showgrid': False,
                                 'showline': False,
                                 'zeroline': False,
                                 'ticks': ""})
        
        fig.update_layout(xaxis2={'domain': [0, .15],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': False,
                                  'ticks': ""})
        
        fig.update_layout(yaxis={'domain': [0, .85],
                                 'mirror': False,
                                 'showgrid': False,
                                 'showline': False,
                                 'zeroline': False,
                                 'showticklabels': False,
                                 'ticks': ""})
        
        fig.update_layout(yaxis2={'domain': [.825, .975],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': False,
                                  'ticks': ""})
        
        fig.show()
            

In [3]:
data_url = '/kaggle/input/team2-dataset/team2_dataset/chap8.xlsx'
label = 'Sales'  # Replace 'YourLabelColumn' with the actual label column name

# Load data
data_loader = dataLoader(data_url, label)

# Get the independent variables (X) and dependent variable (Y)
X, Y = data_loader.detachedData(data_loader.data)

# Perform clustering on the entire dataset
clust = clustering(X)
clust.agglomerative_clustering()
clust.plot_dendrogram_with_heatmap()

Distance Matrix:
[[  0.         187.16415255 213.05405417  79.40214103  57.23748772]
 [187.16415255   0.          37.07411496 107.85434623 139.88148555]
 [213.05405417  37.07411496   0.         134.81205436 167.67760733]
 [ 79.40214103 107.85434623 134.81205436   0.          42.29361654]
 [ 57.23748772 139.88148555 167.67760733  42.29361654   0.        ]]
Clusters: [[0], [1], [2], [3], [4]]
Gộp cụm 1 và 2 với d= 37.07
Clusters đã gộp: [[0], [3], [4], [1, 2]]

Clusters: [[0], [3], [4], [1, 2]]
Gộp cụm 1 và 2 với d= 42.29
Clusters đã gộp: [[0], [1, 2], [3, 4]]

Clusters: [[0], [1, 2], [3, 4]]
Gộp cụm 0 và 2 với d= 57.24
Clusters đã gộp: [[1, 2], [0, 3, 4]]

Clusters: [[1, 2], [0, 3, 4]]
Gộp cụm 0 và 1 với d= 107.85
Clusters đã gộp: [[1, 2, 0, 3, 4]]



                                               ** END!**